# License

In [1]:
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
from __future__ import print_function

def installDeps():
    !pip install numpy scipy
    !pip install resampy tensorflow six
    !pip install youtube_dl
    !pip install ipywidgets
    !pip install pydub
    !pip install tqdm
    !pip install ffmpeg-python
    !apt-get install ffmpeg -y
#installDeps()


from random import shuffle

import numpy as np
import tensorflow as tf
import os
import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioModel import predict, train, accuracy, getCorrectAndIncorrect
from audioInput import getLaughTracks, getNoise, getSamples
from audioUtils import readFolder

def getModel(path):
    files = readFolder('model/%s' % path)
    if len(files) > 0:
        return '%s/%s' % (path, files[0])
    return None

def trainAndSaveAndPredict(test_data, model, number_of_classes = 2, number_of_samples = None, epochs = 5, getData = getLaughTracks, log = True, batch_size = 32, lr = vggish_params.LEARNING_RATE):
    model_name = '%s_%s' % (model, number_of_samples)

    def curriedGetSamples(shuf):
        return getData(number_of_samples = number_of_samples, shuf = shuf, log = log)
    #print('model_name', model_name)
    print('---')
    preds = train(curriedGetSamples, number_of_classes, model_name = model_name, epochs = epochs, batch_size = batch_size)
    
    return predict(getModel('%s' % (model_name)), number_of_classes, test_data)

def printResults(preds, labels): 
    correct = []
    incorrect = []
    print('accuracy', accuracy(preds, labels))
    getCorrectAndIncorrect(preds, labels)
    correct, incorrect = getCorrectAndIncorrect(preds, labels)
    print(correct[0:5])
    print(incorrect[0:5])
    return correct, incorrect
    
def trainForNoise(number_of_samples=5, epochs=5):
    print('training on noise, sin, and constant waves')
    (features, labels) = getNoise(shuf=False, number_of_samples = 2)
    preds = trainAndSaveAndPredict(features, 'noise', number_of_classes = 3, number_of_samples = number_of_samples, epochs = epochs, getData = getNoise, lr = vggish_params.LEARNING_RATE)
    printResults(preds, [0, 0, 1, 1, 2, 2])
    
def trainForLaughter(number_of_samples=None, epochs=5, batch_size = 32, lr = vggish_params.LEARNING_RATE):  
    #print('training on laughter and not laughter')
    (features, labels, chunks) = getSamples(['laughter-test', 'notlaughter-test'], shuf = False)
    preds = trainAndSaveAndPredict(features, 'audio', number_of_classes = 2, epochs = epochs, getData = getLaughTracks, batch_size = batch_size, lr = lr)
    printResults(preds, labels)
    return preds, labels

model_name = '%s_%s' % ('audio', None)
#trainForLaughter(epochs=1, number_of_samples=None, batch_size=32, lr=vggish_params.LEARNING_RATE / 400)
train(getLaughTracks, 2, model_name = model_name, epochs = 1, batch_size = 32)

INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
1 ['samples/laughter/16-lfVsYaxc.wav', 'samples/laughter/3IC76o_lhFw.wav', 'samples/laughter/3LG9A7fUrPs.wav', 'samples/laughter/4B06Bh3i8Ms.wav', 'samples/laughter/4z12ijqiFrY.wav', 'samples/laughter/At6oITvbENo.wav', 'samples/laughter/C80iIItZHFM.wav', 'samples/laughter/DZZORgAVFJw.wav', 'samples/laughter/EzS7y_3GhKA.wav', 'samples/laughter/IlMPU4AVU20.wav', 'samples/laughter/Pz_DMUe4tXc.wav', 'samples/laughter/R1hbmMfoT9c.wav', 'samples/laughter/T_wNZhcw9x8.wav', 'samples/laughter/b7KR9nQbhmQ.wav', 'samples/laughter/bUTY_c6S3VI.wav', 'samples/laughter/fVXpJNZYDH0.wav', 'samples/laughter/iYVO5bUFww0.wav', 'samples/laughter/mbgrSdRs9bQ.wav', 'samples/laughter/nKo-dvnh6J0.wav', 'samples/laughter/rHV09L1_t0g.wav', 'samples/laughter/snl 2 laughter.wav', 'samples/laughter/standup 1 laughter.wav', 'samples/laughter/standup 2 laughter.wav', 'samples/laughter/ultimate_1_a.wav', 'samples/laughter/ultimate_2_a.wav', 'samples/laughte

Exception: file failed: ['samples/laughter/16-lfVsYaxc.wav', 'samples/laughter/3IC76o_lhFw.wav', 'samples/laughter/3LG9A7fUrPs.wav', 'samples/laughter/4B06Bh3i8Ms.wav', 'samples/laughter/4z12ijqiFrY.wav', 'samples/laughter/At6oITvbENo.wav', 'samples/laughter/C80iIItZHFM.wav', 'samples/laughter/DZZORgAVFJw.wav', 'samples/laughter/EzS7y_3GhKA.wav', 'samples/laughter/IlMPU4AVU20.wav', 'samples/laughter/Pz_DMUe4tXc.wav', 'samples/laughter/R1hbmMfoT9c.wav', 'samples/laughter/T_wNZhcw9x8.wav', 'samples/laughter/b7KR9nQbhmQ.wav', 'samples/laughter/bUTY_c6S3VI.wav', 'samples/laughter/fVXpJNZYDH0.wav', 'samples/laughter/iYVO5bUFww0.wav', 'samples/laughter/mbgrSdRs9bQ.wav', 'samples/laughter/nKo-dvnh6J0.wav', 'samples/laughter/rHV09L1_t0g.wav', 'samples/laughter/snl 2 laughter.wav', 'samples/laughter/standup 1 laughter.wav', 'samples/laughter/standup 2 laughter.wav', 'samples/laughter/ultimate_1_a.wav', 'samples/laughter/ultimate_2_a.wav', 'samples/laughter/w0E3rEy4YPQ.wav', 'samples/laughter/yS

In [ ]:
trainForLaughter(epochs=20, number_of_samples=None, batch_size=32, lr=vggish_params.LEARNING_RATE / 400)

Number of features per class {'notlaughter-test': 824, 'laughter-test': 276}
---
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
no cache file available, building one
Number of features per class {'notlaughter': 8383, 'laughter': 9003}


 52%|█████▏    | 251/487 [05:42<05:17,  1.34s/it]

In [2]:
(features, labels) = getSamples(['laughter-test', 'notlaughter-test'], shuf = False, number_of_samples = 1, log=False, use_cache = False, get_vgg = True)
print(features.shape)
print(labels.shape)

(1, 96, 64) [1. 0.]
(1, 96, 64) [0. 1.]
Number of features per class {'laughter-test': 1, 'notlaughter-test': 1}
(2, 96, 64)
(2, 2)


In [3]:
(features, labels) = getSamples(['laughter-test', 'notlaughter-test'], shuf = False, number_of_samples = 1, log=False, get_vgg = False, use_cache = False)
print(features.shape)
print(labels.shape)

(44100,) [1. 0.]
chunks 1
(44100,) [0. 1.]
chunks 1
Number of features per class {'laughter-test': 44100, 'notlaughter-test': 44100}
(88200,)
(2, 2)
